# Modelo Recomendación
Se creará un modelo de recomendación de películas, basado en el dataset recibido, la salida retornará una lista de 5 películas similares recomendadas por el modelo según sus características, dado un título de una película perteneciente al dataset

In [34]:
import pandas as pd

## Preprocesamiento de datos

Carga del dataset

In [35]:
df_premodel = pd.read_parquet('https://github.com/alejocampos1/Henry_PI1_Alejandro-Campos/raw/main/Datasets/Datasets_Limpios/Parquet/movies_tomodel.parquet')
df_genres = pd.read_parquet('https://github.com/alejocampos1/Henry_PI1_Alejandro-Campos/raw/main/Datasets/Datasets_Limpios/Parquet/genres.parquet')

In [36]:
df_premodel.head(2)

,id,title,genres,overview,runtime,production_companies,original_language,spoken_languages,production_countries,popularity,vote_average,vote_count,release_year,weighted_rating
0,862,Toy Story,"[16, 35, 10751]","Led by Woody, Andy's toys live happily in his ...",81.0,[3],en,[en],[US],21.946943,7.7,5415.0,1995,7.68
1,8844,Jumanji,"[12, 14, 10751]",When siblings Judy and Peter discover an encha...,104.0,"[559, 2550, 10201]",en,"[en, fr]",[US],17.015539,6.9,2413.0,1995,6.87


## One-hot label encoding para la columna 'géneros'

Primero, se cambiará el 'id' de género en la columna por su nombre real

In [37]:
# Creamos un diccionario que relacione los IDs de los géneros con sus nombres
generos_dict = dict(zip(df_genres['id'], df_genres['name']))

# Crear una nueva columna en df_premodel con los nombres de los géneros en lugar de los IDs
df_premodel['genres_names'] = df_premodel['genres'].apply(
    lambda x: ["No Data" if item_id == 0 else generos_dict[item_id] for item_id in x]
)

# Eliminar columna duplicada
df_premodel.drop(columns=['genres'], inplace=True)

df_premodel.head(2)

,id,title,overview,runtime,production_companies,original_language,spoken_languages,production_countries,popularity,vote_average,vote_count,release_year,weighted_rating,genres_names
0,862,Toy Story,"Led by Woody, Andy's toys live happily in his ...",81.0,[3],en,[en],[US],21.946943,7.7,5415.0,1995,7.68,"[Animation, Comedy, Family]"
1,8844,Jumanji,When siblings Judy and Peter discover an encha...,104.0,"[559, 2550, 10201]",en,"[en, fr]",[US],17.015539,6.9,2413.0,1995,6.87,"[Adventure, Fantasy, Family]"


Creación del one-hot label enconding

In [40]:
# Expandir las listas de géneros en filas individuales
df_generos_encoded = df_premodel[['id', 'genres_names']].explode('genres_names')

# Get_dummies para crear las columnas one hot
df_generos_encoded = pd.get_dummies(df_generos_encoded, columns=['genres_names'], prefix='', prefix_sep='')

# Nuevo Dataframe agrupado

df_generos_encoded = df_generos_encoded.groupby('id').sum().reset_index()

# Hacer un merge de df_premodel con df_generos_encoded usando la columna 'id'
df_merged = pd.merge(df_premodel, df_generos_encoded, on='id', how='left')

# Verificar el resultado
df_merged.head()


,id,title,overview,runtime,production_companies,original_language,spoken_languages,production_countries,popularity,vote_average,...,Horror,Music,Mystery,No Data,Romance,Science Fiction,TV Movie,Thriller,War,Western
0,862,Toy Story,"Led by Woody, Andy's toys live happily in his ...",81.0,[3],en,[en],[US],21.946943,7.7,...,0,0,0,0,0,0,0,0,0,0
1,8844,Jumanji,When siblings Judy and Peter discover an encha...,104.0,"[559, 2550, 10201]",en,"[en, fr]",[US],17.015539,6.9,...,0,0,0,0,0,0,0,0,0,0
2,15602,Grumpier Old Men,A family wedding reignites the ancient feud be...,101.0,"[6194, 19464]",en,[en],[US],11.712900,6.5,...,0,0,0,0,1,0,0,0,0,0
3,31357,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",127.0,[306],en,[en],[US],3.859495,6.1,...,0,0,0,0,1,0,0,0,0,0
4,11862,Father of the Bride Part II,Just when George Banks has recovered from his ...,106.0,"[5842, 9195]",en,[en],[US],8.387519,5.7,...,0,0,0,0,0,0,0,0,0,0
